In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import TargetEncoder

# Load the dataset
dataset = pd.read_csv('datasets/dataset_final_3_target_encoding.csv')

# Prepare features and target
X = dataset.drop('TOTAL_FIRE_SIZE', axis=1).values
y = dataset['TOTAL_FIRE_SIZE'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Build a simple neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.001,
#     decay_steps=10000,
#     decay_rate=0.9
# )
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=1000, batch_size=64, validation_split=0.2)
y_pred = model.predict(X_test).flatten()

def calc_log_clamped_score_np(y_true, y_pred):
    eps = 1e-15
    y_pred = np.maximum(y_pred, eps)
    y_true = np.maximum(y_true, eps)
    log_errors = np.abs(np.log(y_pred / y_true))
    log_errors_clamped = np.minimum(log_errors, 10.0)
    return np.mean(log_errors_clamped)

score = calc_log_clamped_score_np(y_test, y_pred)
print("Score:", score)

Epoch 1/1000


/home/franciszek-latala/miniconda3/envs/nlp/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7848288768.0000 - val_loss: 1419364864.0000
Epoch 2/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6998632448.0000 - val_loss: 1550191104.0000
Epoch 3/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9395015680.0000 - val_loss: 1561607680.0000
Epoch 4/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4688150528.0000 - val_loss: 1598772224.0000
Epoch 5/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4648682496.0000 - val_loss: 1710465664.0000
Epoch 6/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6230769664.0000 - val_loss: 1623209216.0000
Epoch 7/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8460109824.0000 - val_loss: 1599565312.0000
Epoch 8/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12581154816.0000 - val_loss: 1556478208.0000
Epoch 9/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14874990592.0000 - val_loss: 1439543424.0000
Epoch 10/1000
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4416228864.000

In [17]:
# model.save('datasets/best_model.h5')
model.save('models/best_model_2.keras')